In [14]:
from pyscf import ao2mo, gto, scf, fci, tools
import numpy as np
from functools import reduce
np.set_printoptions(precision=3, linewidth=120, suppress=True)

In [3]:
mol = gto.M(
    atom = 'Be  0.0      0.0      0.66242; \
            Be  0.0      0.0     -0.66242;',
    basis="sto-3g",
    spin = 2,
    verbose = 4
)


# omega = 0.5
# mf = dft.RKS(mol)
# mf._numint.libxc = dft.xcfun
# mf.xc = f'ldaerf + lr_hf({omega})'
# mf.omega = omega

mf = scf.UHF(mol)
mf.kernel()


System: uname_result(system='Linux', node='pauli', release='6.6.1-arch1-1', version='#1 SMP PREEMPT_DYNAMIC Wed, 08 Nov 2023 16:05:38 +0000', machine='x86_64')  Threads 16
Python 3.10.10 (main, Mar 21 2023, 18:45:11) [GCC 11.2.0]
numpy 1.23.4  scipy 1.9.3
Date: Fri Nov 10 17:28:46 2023
PySCF version 2.1.1
PySCF path  /home/stefano/Miniconda/envs/qiskit/lib/python3.10/site-packages/pyscf

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 2
[INPUT] num. electrons = 8
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 2
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 Be     0.000000000000   0.000000000000   0.662420000000 AA    0.000000000000   0.000000000000   1.251792379434 Bohr   0.0
[INPUT]  2 Be     0.000000000000   0.000000000000  -0.662420000000 AA    0.000000000000   0.000000000000  -1.2517923794

-28.53808623907689

In [4]:
# density matrices
dm1a,dm1b = mf.make_rdm1()
dm2aa,dm2ab,dm2bb = mf.make_rdm2()
# overlap
S = mf.get_ovlp()

In [5]:
fci.spin_op.spin_square_general(dm1a, dm1b, dm2aa, dm2ab, dm2bb, np.eye(mf.mo_coeff[0].shape[0]), S)

(2.1286099983509392, 3.0845485882708603)

In [6]:
s,U = np.linalg.eigh(S)
X = U @ np.diag(1/np.sqrt(s)) @ U.T
invX = U @ np.diag(np.sqrt(s)) @ U.T

In [7]:
fci.spin_op.spin_square_general(dm1a, dm1b, dm2aa, dm2ab, dm2bb, (invX,invX))

(2.128609998350947, 3.0845485882708656)

In [8]:
C_a = mf.mo_coeff[0]
C_b = mf.mo_coeff[1]
SC_a = S@mf.mo_coeff[0]
SC_b = S@mf.mo_coeff[1]
Cp_a = np.linalg.inv(X) @ mf.mo_coeff[0]
Cp_b = np.linalg.inv(X) @ mf.mo_coeff[1]

In [9]:
dm1a_mo = np.einsum("pq,pa,qb->ab", dm1a, SC_a, SC_a)
dm1b_mo = np.einsum("pq,pa,qb->ab", dm1b, SC_b, SC_b)
dm2ab_mo = np.einsum("pqrs,pa,qb,rc,sd->abcd", dm2ab, SC_a, SC_a, SC_b, SC_b)
dm2aa_mo = np.einsum("pqrs,pa,qb,rc,sd->abcd", dm2aa, SC_a, SC_a, SC_a, SC_a)
dm2bb_mo = np.einsum("pqrs,pa,qb,rc,sd->abcd", dm2bb, SC_b, SC_b, SC_b, SC_b)

In [10]:
fci.spin_op.spin_square_general(dm1a_mo, dm1b_mo, dm2aa_mo, dm2ab_mo, dm2bb_mo, (C_a,C_b), S)

(2.128609998350922, 3.084548588270849)

In [11]:
fci.spin_op.spin_square_general(dm1a_mo, dm1b_mo, dm2aa_mo, dm2ab_mo, dm2bb_mo, (Cp_a,Cp_b))

(2.12860999835092, 3.084548588270848)

In [12]:
myfci = fci.FCI(mf)
myfci.verbose = 5
E,CI = myfci.kernel()

davidson 0 1  |r|= 0.199  e= [-34.937]  max|de|= -34.9  lindep= 0.00463
davidson 1 2  |r|= 0.0896  e= [-34.984]  max|de|= -0.0472  lindep= 0.531
davidson 2 3  |r|= 0.0509  e= [-34.993]  max|de|= -0.00956  lindep= 0.735
davidson 3 4  |r|= 0.0185  e= [-34.996]  max|de|= -0.00224  lindep= 0.892
davidson 4 5  |r|= 0.00804  e= [-34.996]  max|de|= -0.000295  lindep= 0.92
davidson 5 6  |r|= 0.00537  e= [-34.996]  max|de|= -9.43e-05  lindep= 0.885
davidson 6 7  |r|= 0.00148  e= [-34.996]  max|de|= -1.93e-05  lindep= 0.896
davidson 7 8  |r|= 0.000336  e= [-34.996]  max|de|= -1.42e-06  lindep= 0.944
davidson 8 9  |r|= 7.64e-05  e= [-34.996]  max|de|= -6.42e-08  lindep= 0.94
davidson 9 10  |r|= 2.33e-05  e= [-34.996]  max|de|= -2.27e-09  lindep= 0.687
davidson 10 11  |r|= 6.2e-06  e= [-34.996]  max|de|= -3.25e-10  lindep= 0.816
root 0 converged  |r|= 4.54e-06  e= -34.99597142886025  max|de|= -3.68e-11
converged 11 12  |r|= 4.54e-06  e= [-34.996]  max|de|= -3.68e-11


In [13]:
# the FCI wave function fixes the spin contamination from UHF
fci.spin_op.spin_square(CI, 10, (5,3), mf.mo_coeff, ovlp=S)

(2.0000000000523257, 3.0000000000348837)

In [16]:
# we need to pass from cp2k to qiskit (Cp_a, Cp_b) to compute the correct spin square
fci.spin_op.spin_square(CI, 10, (5,3), (Cp_a,Cp_b))

(2.000000000052323, 3.000000000034882)

In [15]:
# careful that pyscf does not do the right thing if spin_square is called from the fci object
myfci.spin_square(CI, 10, (5,3))

(2.216261475737743, 3.140867062285663)

# Testing for RHF

In [33]:
mol.spin = 0
mf = scf.RHF(mol)
mf.kernel()



******** <class 'pyscf.scf.hf.RHF'> ********
method = RHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = 50
direct_scf = True
direct_scf_tol = 1e-13
chkfile to save SCF result = /home/stefano/Projects/embedding/periodic-embedding-data/test/tmp8emrq51t
max_memory 4000 MB (current use 220 MB)
Set gradient conv threshold to 3.16228e-05
init E= -29.236868744872
  HOMO = -0.0323059292961095  LUMO = 0.0176364828233575
cycle= 1 E= -28.3975973537882  delta_E= 0.839  |g|= 0.101  |ddm|= 2.11
  HOMO = -0.164485019415418  LUMO = 0.0343019819561644
cycle= 2 E= -28.4086353015049  delta_E= -0.011  |g|= 0.0383  |ddm|= 0.276
  HOMO = -0.148674896796845  LUMO = 0.0561029080098722
cycle= 3 E= -28.4105416505723  delta_E= -0.00191  |g|= 0.00794  |ddm|= 0.134
  HOMO = -0.145862107703947  LUMO = 0.0588092318823599
cycle= 4 E= -28.4106039047415  delta_E= -

-28.410604055570907

In [35]:
# density matrices
dm1 = mf.make_rdm1()
dm2 = mf.make_rdm2()
# overlap
S = mf.get_ovlp()

In [39]:
s,U = np.linalg.eigh(S)
X = U @ np.diag(1/np.sqrt(s)) @ U.T
invX = U @ np.diag(np.sqrt(s)) @ U.T

In [40]:
C_a = mf.mo_coeff
C_b = mf.mo_coeff
SC_a = S@mf.mo_coeff
SC_b = S@mf.mo_coeff
Cp_a = np.linalg.inv(X) @ mf.mo_coeff
Cp_b = np.linalg.inv(X) @ mf.mo_coeff

In [54]:
dm1_mo = np.einsum("pq,pa,qb->ab", dm1, SC_a, SC_a)
dm1a_mo = 0.5*dm1_mo
dm1b_mo = 0.5*dm1_mo
dm2aa_mo = (np.einsum('ij,kl->ijkl', dm1a_mo, dm1a_mo)
         - np.einsum('ij,kl->iklj', dm1a_mo, dm1a_mo))
dm2bb_mo = (np.einsum('ij,kl->ijkl', dm1b_mo, dm1b_mo)
         - np.einsum('ij,kl->iklj', dm1b_mo, dm1b_mo))
dm2ab_mo = np.einsum('ij,kl->ijkl', dm1a_mo, dm1b_mo)

In [55]:
fci.spin_op.spin_square_general(dm1a_mo, dm1b_mo, dm2aa_mo, dm2ab_mo, dm2bb_mo, (C_a,C_b), S)

(5.773159728050814e-15, 1.0000000000000115)

In [56]:
fci.spin_op.spin_square_general(dm1a_mo, dm1b_mo, dm2aa_mo, dm2ab_mo, dm2bb_mo, (Cp_a,Cp_b))

(8.881784197001252e-16, 1.0000000000000018)

In [59]:
myfci = fci.FCI(mf)
myfci.verbose = 5
E,CI = myfci.kernel()

davidson 0 1  |r|= 0.206  e= [-34.931]  max|de|= -34.9  lindep= 0.402
davidson 1 2  |r|= 0.195  e= [-34.951]  max|de|= -0.0199  lindep= 0.125
davidson 2 3  |r|= 0.0659  e= [-34.989]  max|de|= -0.0383  lindep= 0.686
davidson 3 4  |r|= 0.0375  e= [-34.995]  max|de|= -0.00543  lindep= 0.793
davidson 4 5  |r|= 0.0142  e= [-34.996]  max|de|= -0.00113  lindep= 0.892
davidson 5 6  |r|= 0.00415  e= [-34.996]  max|de|= -0.00013  lindep= 0.925
davidson 6 7  |r|= 0.00131  e= [-34.996]  max|de|= -1.33e-05  lindep= 0.936
davidson 7 8  |r|= 0.000544  e= [-34.996]  max|de|= -1.41e-06  lindep= 0.967
davidson 8 9  |r|= 0.000201  e= [-34.996]  max|de|= -2.56e-07  lindep= 0.932
davidson 9 10  |r|= 5.18e-05  e= [-34.996]  max|de|= -2.04e-08  lindep= 0.971
davidson 10 11  |r|= 1.76e-05  e= [-34.996]  max|de|= -1.48e-09  lindep= 0.922
davidson 11 12  |r|= 5.07e-06  e= [-34.996]  max|de|= -1.78e-10  lindep= 0.927
root 0 converged  |r|= 5.07e-06  e= -34.995971428916285  max|de|= -1.42e-14
converged 12 1  |r|=

In [69]:
def log_ci_states(fci_solver, uhf, thresh=1e-6):
    norb = fci_solver.norb
    nel_a, nel_b = fci_solver.nelec
    occslst_a = fci.cistring._gen_occslst(range(norb), nel_a)
    occslst_b = fci.cistring._gen_occslst(range(norb), nel_b)

    for root in range(fci_solver.nroots):
        if fci_solver.nroots == 1:
            ci_vector = fci_solver.ci
        else:
            ci_vector = fci_solver.ci[root]

        print(
            f"Logging CI vectors and coefficients > {thresh} for root number {root}:"
        )
        # log S^2
        if not uhf:
            spin_square = fci_solver.spin_square(ci_vector, norb, (nel_a, nel_b))
            print("This root has S^2 = %s" % spin_square[0])
        else:
            print("Cannot compute S^2 for unrestricted spin right now... Stay tuned!")

        pad = 4 + norb
        print(f'  {"Conf": <{pad}} CI coefficients')
        for i, occsa in enumerate(occslst_a):
            for j, occsb in enumerate(occslst_b):
                if abs(ci_vector[i, j]) < thresh:
                    continue
                # generate the CI string and log it
                occ = ""
                for k in range(norb):
                    if k in occsa and k in occsb:
                        occ += "2"
                    elif k in occsa and k not in occsb:
                        occ += "u"
                    elif k not in occsa and k in occsb:
                        occ += "d"
                    else:
                        occ += "0"
                print("  %s     %+.8f" % (occ, ci_vector[i, j]))

In [83]:
# Careful that the FCI might find states of different spin symmetry!
log_ci_states(myfci, False, 0.1)

Logging CI vectors and coefficients > 0.1 for root number 0:
This root has S^2 = 1.9999999996460383
  Conf           CI coefficients
  2220ud0000     +0.66430835
  2220du0000     -0.66430738


In [76]:
fci.spin_op.spin_square(CI, 10, (4,4), mf.mo_coeff, ovlp=S)

(1.999999999646045, 2.99999999976403)

In [80]:
# we need to pass from cp2k to qiskit (Cp_a, Cp_b) to compute the correct spin square, but for a RHF MOs we don't really need them, because the call below works fine
fci.spin_op.spin_square(CI, 10, (4,4), (Cp_a,Cp_b))

(1.9999999996460405, 2.999999999764027)

In [79]:
# pyscf does the right thing if spin_square is called from the fci object created with a RHF mean-field
myfci.spin_square(CI, 10, (4,4))

(1.9999999996460383, 2.9999999997640256)

# Check that the alpha-beta MO orbital overlap is the same in CP2K and pyscf

This calculation below gives roughly the same as a corresponding calculation in CP2K. I am a bit puzzled by the sign, especially in the diagonal. In CP2K we get a different sign only on a single element of the diagonal, I am not sure whether this is problematic.

In [15]:
R = 1.1

mol = gto.M(
    atom = f'C 0.0 0.0 0.0; \
             N 0.0 0.0 {R};',
    basis='sto-3g',
    spin=1,
    charge=0,
    verbose=3
)

# omega = 0.5
# mf_o2 = dft.UKS(mol)
# mf_o2._numint.libxc = dft.xcfun
# mf_o2.xc = f'ldaerf + lr_hf({omega})'
# mf_o2.omega = omega
# mf_o2.kernel()

mf = scf.UHF(mol)
mf.kernel()

tools.molden.dump_scf(mf, 'cn.molden')

converged SCF energy = -90.9967795701639  <S^2> = 0.98704588  2S+1 = 2.2244513


In [21]:
S = mf.get_ovlp()
Ca = mf.mo_coeff[0][:, 6:9]
Cb = mf.mo_coeff[1][:, 6:9]

In [25]:
Ca.transpose().dot(S).dot(Cb)

array([[ 0.534, -0.   ,  0.   ],
       [-0.   , -0.919, -0.193],
       [-0.   ,  0.193, -0.919]])